In [0]:
import dlt
import requests
from pyspark.sql.functions import col, count, explode, collect_list, desc, first, round, sum as sum_spark

@dlt.table(
    name="generos",
    comment="dados da bigtables gerando uma tabela genero"
)
def playtime_silver():
    df = dlt.read("steamdatabricks_workspace.gold.bigtable")

    df_proporcao = dlt.read("steamdatabricks_workspace.gold.playtime")

    genres = df.select('appid', 'id_genres', 'description_genres').distinct()

    genres = genres.withColumnRenamed("description_genres", "genres")

    df_generos_por_app = (
        genres.groupBy("genres")
            .agg(
                count("appid").alias("qtd_games"),
                collect_list("appid").alias("appids")
            )
            .orderBy(desc("qtd_games"))
    )

    df_exploded = df_generos_por_app.select("genres", "qtd_games", explode("appids").alias("appid"))

    df_joined = df_exploded.join(df_proporcao.select("appid", "playtime_forever"), on="appid", how="left")

    df_generos_por_app_join = (
        df_joined.groupBy("genres")
            .agg(
                first("qtd_games").alias("qtd_games"),
                round(sum_spark("playtime_forever"), 2).alias("playtime_total"),
                collect_list("appid").alias("appids")
            )
            .orderBy(desc("playtime_total"))
    )

    return df_generos_por_app_join